In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
from datetime import datetime, timezone
import hashlib
import time
import random

BASE_URL = "https://www.eluniversal.com.mx/minuto-x-minuto"

def get_headers():
    return {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/123.0.0.0 Safari/537.36"
        )
    }

def generate_id(url, title):
    return hashlib.md5(f"{url}_{title}".encode("utf-8")).hexdigest()[:12]

def scrape_eluniversal(max_pages=5):
    articles = []
    seen_urls = set()

    for page in range(1, max_pages + 1):
        url = BASE_URL if page == 1 else f"{BASE_URL}/todos/{page}"
        print(f"Descargando página {page}: {url}")

        try:
            response = requests.get(url, headers=get_headers(), timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")

            news_items = soup.select('div[class*="listing-item"] h2 a, article h2 a')
            if not news_items:
                print("No se encontraron noticias en esta página.")
                continue

            for item in news_items:
                link = item.get("href")
                if not link:
                    continue
                if not link.startswith("http"):
                    link = f"https://www.eluniversal.com.mx{link}"
                if link in seen_urls:
                    continue
                seen_urls.add(link)

                title = item.get_text(strip=True)
                if not title:
                    continue

                try:
                    art_resp = requests.get(link, headers=get_headers(), timeout=10)
                    art_resp.raise_for_status()
                    art_soup = BeautifulSoup(art_resp.content, "html.parser")

                    fecha_elem = art_soup.select_one("time")
                    fecha = fecha_elem.get("datetime", datetime.now().strftime("%Y-%m-%d")) if fecha_elem else datetime.now().strftime("%Y-%m-%d")

                    author_selectors = [
                        ".sc__author-nota",
                        ".author",
                        ".byline"
                    ]
                    autor = "El Universal Staff"
                    for sel in author_selectors:
                        author_elem = art_soup.select_one(sel)
                        if author_elem and author_elem.get_text(strip=True):
                            autor = author_elem.get_text(strip=True)
                            break

                except Exception:
                    fecha = datetime.now().strftime("%Y-%m-%d")
                    autor = "El Universal Staff"

                article = {
                    "id": generate_id(link, title),
                    "titulo": title,
                    "fecha": fecha,
                    "url": link,
                    "fuente": "El Universal",
                    "autor": autor,
                    "capturado_ts": datetime.now(timezone.utc).isoformat()
                }
                articles.append(article)
                print(f"Noticia: {title[:50]}...")

            time.sleep(random.uniform(1, 2))

        except requests.RequestException as e:
            print(f"Error descargando {url}: {e}")
            continue

    print(f"Total de noticias extraídas: {len(articles)}")
    return articles

def save_jsonl(data, filename="data/raw/noticias.jsonl"):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w", encoding="utf-8") as f:
        for record in data:
            json.dump(record, f, ensure_ascii=False)
            f.write("\n")
    print(f"Guardadas {len(data)} noticias en {filename}")

def profile_data(articles, report_path="reports/perfilado.md"):
    total = len(articles)
    fields = ["id", "titulo", "fecha", "url", "fuente", "autor", "capturado_ts"]
    null_counts = {f: 0 for f in fields}
    urls = []
    ids = []

    for r in articles:
        for f in fields:
            if not r.get(f):
                null_counts[f] += 1
        urls.append(r.get("url"))
        ids.append(r.get("id"))

    dup_urls = len(urls) - len(set(urls))
    dup_ids = len(ids) - len(set(ids))

    lines = []
    lines.append(f"# Perfilado de Calidad - {datetime.now(timezone.utc).isoformat()}\n")
    lines.append(f"- Total de registros: {total}\n")
    lines.append(f"- Duplicados por URL: {dup_urls}\n")
    lines.append(f"- Duplicados por ID: {dup_ids}\n")
    lines.append("\n## Valores nulos por campo:\n")
    for f in fields:
        lines.append(f"- {f}: {null_counts[f]} ({null_counts[f]/total:.2%})\n")

    os.makedirs(os.path.dirname(report_path), exist_ok=True)
    with open(report_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    print(f"Reporte de perfilado generado en {report_path}")

def main():
    print(f"Iniciando scraping de {BASE_URL}")
    articles = scrape_eluniversal(max_pages=5)
    if articles:
        save_jsonl(articles)
        profile_data(articles)
    else:
        print("No se obtuvieron noticias.")

if __name__ == "__main__":
    main()

Iniciando scraping de https://www.eluniversal.com.mx/minuto-x-minuto
Descargando página 1: https://www.eluniversal.com.mx/minuto-x-minuto
Noticia: Ciudadana estadounidense es baleada por agentes de...
Noticia: Vinculan a proceso a exfuncionario de Contraloría ...
Noticia: Trump dice que Israel acordó línea de retirada ini...
Noticia: Mark Sánchez, exjugador de la NFL, fue apuñalado e...
Noticia: "La casa de los famosos México": ¿Cómo votar por t...
Noticia: Guadalajara ya se alista para recibir los partidos...
Noticia: Embajador de EU en México celebra detención de ope...
Noticia: Demócratas extenderán el cierre de gobierno en EU ...
Noticia: Series Netflix: ¿cuántos capítulos tiene "Monstruo...
Noticia: Paro de operadores del transporte en Mérida activa...
Descargando página 2: https://www.eluniversal.com.mx/minuto-x-minuto/todos/2
Noticia: Sean "Diddy" Combs, Kevin Spacey, Michael Jackson ...
Noticia: Gobierno de Trump ofrece 2 mil 500 dólares a niños...
Noticia: Tigres vs Cruz Azul: